In [ ]:
#the library for the code
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import *
from time import time
import itertools
import gc
from sklearn import svm

In [ ]:
# the random walk kernel
def kernel1(a1):
    a0=np.zeros((a1.shape[0],a1.shape[0]))
    mylist=list(np.unique(np.nonzero(a1)))
    for i in mylist:
        a0[i,i]=1
    a2=a1*a1
    a3=a2*a1  
    a4=a3*a1
    F=a0+a1+a2+a3+a4;
    return F


In [ ]:
# the function to build graphlets
def graphles(data,f,flag=1):
    df1=pd.DataFrame()
    mylist = []
        
    #the basis graph includes all the elements in the train dataset
    D = nx.DiGraph()
    D.add_nodes_from([node for node in f['A']])
    D.add_nodes_from([node for node in f['B']])
    D.add_nodes_from([node for node in f['C']])
    D.add_nodes_from([node for node in f['D']])
    D.add_nodes_from([node for node in f['E']])
    D.add_nodes_from([node for node in f['F']])
    a = list(set(data['A']))
    for i in a:
        
        #activity graphlets
        G = nx.DiGraph()
        df=data[data['A']==i]
        G.add_nodes_from([node for node in df['A']])
        G.add_nodes_from([node for node in df['B']])
        G.add_nodes_from([node for node in df['C']])
        G.add_nodes_from([node for node in df['D']])
        G.add_nodes_from([node for node in df['E']])
        G.add_nodes_from([node for node in df['F']])
        G.add_edges_from([edge for edge in zip(df['A'],df['B'])])
        G.add_edges_from([edge for edge in zip(df['B'],df['C'])])
        G.add_edges_from([edge for edge in zip(df['c'],df['D'])])
        G.add_edges_from([edge for edge in zip(df['D'],df['E'])])
        G.add_edges_from([edge for edge in zip(df['E'],df['F'])])                
        
        #significant nodes
        b = list(set(df['B']))
        c = list(set(df['C']))
        e = list(set(df['E']))
        f = list(set(df['F']))
        
        TNode=[];
        tmp='A';
        com=2;
        for j in b:
            x=int(G.in_degree(j))
            y=int(G.out_degree(j))
            if (x+y>com):
                tmp=j
                com=x+y
        TNode.append(tmp)
        com=1;
        for j in c:
            y=int(G.out_degree(j))
            if (y>com):
                tmp=j
                com=y
        TNode.append(tmp)
        com=1;
        for j in f:
            y=int(G.in_degree(j))
            if (y>com):
                tmp=j
                com=y
        TNode.append(tmp)        
        com=2;
        for j in e:
            x=int(G.in_degree(j))
            y=int(G.out_degree(j))
            if (x+y>com):
                tmp=j
                com=x+y
        TNode.append(tmp)  
        G.clear()
        del G
        
        #profile graphlets
        df2=df;
        for index, row  in df.iterrows():
            tmp=0
            if (row["B"]==TNode[0]):
                tmp=tmp+1
            if (row["C"]==TNode[1]):
                tmp=tmp+1
            if (row["F"]==TNode[2]):
                tmp=tmp+1
            if (row["E"]==TNode[3]):
                tmp=tmp+1
            if (tmp<2):
                df2.drop(index)
        
        D.add_edges_from([edge for edge in zip(df2['A'],df['B'])])
        D.add_edges_from([edge for edge in zip(df2['B'],df['C'])])
        D.add_edges_from([edge for edge in zip(df2['C'],df['D'])])
        D.add_edges_from([edge for edge in zip(df2['D'],df['E'])])
        D.add_edges_from([edge for edge in zip(df2['E'],df['F'])])
        
        #calculate the feature values for each graphlet
        tm=[]
        if flag==1: # use the random walk kernel
            a1=nx.adj_matrix(D)
            cd=kernel1(a1)
            tm=cd.flatten().tolist()
        else: #use the shortest path kernel
            cd=nx.floyd_warshall_numpy(D)
            tm=cd.flatten().tolist()
        D.remove_edges_from(list(D.edges()))
        out = list(itertools.chain.from_iterable(tm))
        mylist.append(out)
        
        df1=pd.concat([df1, df2], axis=0, ignore_index=True) #the significant records
        
    del D
    return df1,mylist
    

In [ ]:
# make the huge dataset into several blocks and easy to compute
def seperate(dt,n):
    dtgrouped=dt.groupby(dt['A'])
    t=dtgrouped.size()
    s=0;
    sep=[];
    for i in range(0,len(t)):
        s=s+t[i]
        if (s>n):
            sep.append(s)
            s=0
    return sep

In [ ]:
def runsep(df1,df,n=500,flag=1):
    sub=pd.DataFrame()
    mylist = []
    num=seperate(df1,n)
    s=0
    t=0
    if num==[]:
        sub,mylist=graphles(df1,df,flag)
    else:
        for i in range(0,(len(num)+1)):
            if (i==len(num)):
                dfcut=df1[s:]
            else:
                t=s+num[i]
                dfcut=df1[s:t]
            sub1,list1=graphles(dfcut,df,flag)
            sub=pd.concat([sub,sub1])
            mylist=mylist+list1
            s=t
    return sub,mylist
        

In [ ]:
df = pd.read_csv("annotated-trace.csv",names=['A','C','B','D','E','Label'])
df['A'] = 'a' + df['A'].astype(str)
df['B'] = 'b' + df['B'].astype(str)
df['F'] = 'f' + df['C'].astype(str)
df['C'] = 'c' + df['C'].astype(str)
df['D'] = 'd' + df['D'].astype(str)
df['E'] = 'e' + df['E'].astype(str)
df['Label'] = df['Label'].astype(str)
df=df.sort_values(by=['Label', 'A','B','C','E'],ascending=False)
df1=df[df['Label']=='normal']
df2=df[df['Label']=='anomaly']


In [ ]:
sub1,f1=runsep(df1,df,500)
sub2,f2=runsep(df2,df)

In [ ]:
len(f1)+len(f2)

In [ ]:
X=f1+f2
t=np.append(np.zeros(len(f1)),np.ones(len(f2)))
y=np.transpose([t])

In [ ]:
classif = svm.SVC(kernel='linear')

start = time()
classif.fit(X, y.ravel())
t11=(time() - start)

In [ ]:
def my_kernel(X, Y):
   return np.dot(X, Y.T)

In [ ]:
clf = svm.SVC(kernel=my_kernel)
start = time()
clf.fit(X, y.ravel())
t13=(time() - start)

In [ ]:
print(t11,"----time----",t13)

In [ ]:
dt = pd.read_csv("not-annotated-trace.csv",names=['A','C','B','D','E'])
dt['A'] = 'a' + dt['A'].astype(str)
dt['B'] = 'b' + dt['B'].astype(str)
dt['F'] = 'f' + dt['C'].astype(str)
dt['C'] = 'c' + dt['C'].astype(str)
dt['D'] = 'd' + dt['D'].astype(str)
dt['E'] = 'e' + dt['E'].astype(str)
dt=dt.sort_values(by=['A','B','C','E'])
dt1=dt
dt=dt[dt['A'].isin(df['A'])]
dt=dt[dt['B'].isin(df['B'])]
dt=dt[dt['C'].isin(df['C'])]
dt=dt[dt['D'].isin(df['D'])]
dt=dt[dt['E'].isin(df['E'])]
dt=dt[dt['F'].isin(df['F'])]


In [ ]:
subt1=dt1.loc[dt1.index.difference(dt.index)]
subt,ft=runsep(dt,df)
subt1,q=runsep(subt1,df)
subt=pd.concat([subt,subt1])

In [ ]:
yt=classif.predict(ft)
yt=np.append(yt,np.ones(len(set(subt1['A']))))
index=0;
listsub=list(set(subt['A']))
for i in yt:
    if (i==1):
        idc=listsub[index]
        gp=subt[subt['A']==idc]
        D = nx.DiGraph()
        D.add_edges_from([edge for edge in zip(gp['A'],gp['B'])])
        D.add_edges_from([edge for edge in zip(gp['B'],gp['C'])])
        D.add_edges_from([edge for edge in zip(gp['C'],gp['D'])])
        D.add_edges_from([edge for edge in zip(gp['D'],gp['E'])])
        D.add_edges_from([edge for edge in zip(gp['E'],gp['F'])])
        plt.clf()
        nx.draw(D,with_labels=True, font_weight='bold')
        plt.savefig("examples%d.jpg"%(index))
    index=index+1


## Classification evaluation

In [ ]:
# Split dataset in train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [ ]:

from sklearn import svm
clf = svm.SVC(kernel=my_kernel)
clf.fit(X_train, y_train.ravel())

In [ ]:

from sklearn.metrics import classification_report

print(classification_report(y_test, clf.predict(X_test)))

In [ ]:
from sklearn.metrics import confusion_matrix
# from: https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    import itertools

    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

# Class_names
class_names = ['0', '1']

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, clf.predict(X_test))
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()


# Shortest path method

We have already made the code in the upper code. To use it, simply to reload the dataset and make graphlets with the command below. The other steps are the same.

In [ ]:
sub1,f1=runsep(df1[:],df,500,0)
sub2,f2=runsep(df2,df,500,0)
subt,ft=runsep(dt1,df,500,0)
